In [773]:
# !python -m pip uninstall scikit-learn --yes

In [774]:
# pip install scikit-learn==1.2.2 --user

In [775]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [776]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [777]:
# import sklearn
# print(sklearn.__version__)

In [778]:
from imblearn.over_sampling import SMOTENC
from sklearn.utils import resample

In [779]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [780]:
file_path = './data/data_iu_proccessed_p3.csv'
# file_path = './data/data_iu_proccessed_p3_v2.csv'

# Read the CSV file with specified data types
df = pd.read_csv(file_path)

In [781]:
df.head(5)

,Unnamed: 0,TenMH,MaMH,Major,SoTinChi,SiSoTKB,ClassifyAs,TKB_LT2,TKB_LT3,TKB_LT4,...,TKB_LT7,TKB_TH2,TKB_TH3,TKB_TH4,TKB_TH5,TKB_TH6,TKB_TH7,HK,GVLT,GVTH
0,0,Kinh tế hành vi và chính sách công,PM5110,BA BA,2,50.0,4,0,0,0,...,0,0,0,0,0,0,0,2,Nguyen Van Phuong,unassigned
1,1,Kinh tế hành vi và chính sách công,PM5110,BA BA,2,61.0,2,0,0,0,...,0,0,0,0,0,0,0,2,Nguyen Van Phuong,unassigned
2,2,Kinh tế hành vi và chính sách công,PM5110,BA BA,2,14.0,1,0,0,0,...,7,0,0,0,0,0,0,3,Nguyen Van Phuong,unassigned
3,3,Kinh tế lượng và phân tích chính sách,PM5214,BA BA,2,50.0,4,0,0,0,...,0,0,0,0,0,0,0,2,Nguyen Phuong Anh,unassigned
4,4,Kỹ năng lãnh đạo trong khu vực công,PM5212,BA BA,2,41.0,2,0,0,0,...,7,0,0,0,0,0,0,1,Trinh Viet Dung,unassigned


In [782]:
# Remove the specified columns
columns_to_remove = ['Unnamed: 0', 'TenMH']  # Columns to be removed
df.drop(columns=columns_to_remove, inplace=True)
df.dtypes

MaMH           object
Major          object
SoTinChi        int64
SiSoTKB       float64
ClassifyAs      int64
TKB_LT2         int64
TKB_LT3         int64
TKB_LT4         int64
TKB_LT5         int64
TKB_LT6         int64
TKB_LT7         int64
TKB_TH2         int64
TKB_TH3         int64
TKB_TH4         int64
TKB_TH5         int64
TKB_TH6         int64
TKB_TH7         int64
HK              int64
GVLT           object
GVTH           object
dtype: object

In [783]:
df.shape

(8341, 20)

In [784]:
#-----------4 Classes--------------
# #Create a dictionary to map values
# class_mapping = {1: 'Class1', 2: 'Class2', 3: 'Class3', 4: 'Class4'}

# # Map the values in the 'ClassifyAs' column
# df['ClassifyAs'] = df['ClassifyAs'].map(class_mapping)

# #-----------2 Classes (Over90)--------------
# # Create a dictionary to map values
# class_mapping = {1: 'Class1', 2: 'Class1', 3: 'Class2', 4: 'Class2'}

# # Map the values in the 'ClassifyAs' column
# df['ClassifyAs'] = df['ClassifyAs'].map(class_mapping)

#---------- 2 Classes (TR89-Spread)-------------
# Create a dictionary to map values
class_mapping = {1: 'Class1', 2: 'Class2', 3: 'Class1', 4: 'Class2'}

# Map the values in the 'ClassifyAs' column
df['ClassifyAs'] = df['ClassifyAs'].map(class_mapping)

#---------- 2 Classes (High vs Low priority course)-------------
# # Create a dictionary to map values
# class_mapping = {1: 'Class1', 2: 'Class2', 3: 'Class2', 4: 'Class2'}

# # Map the values in the 'ClassifyAs' column
# df['ClassifyAs'] = df['ClassifyAs'].map(class_mapping)

In [785]:
# Count the occurrences of each class
class_counts = df['ClassifyAs'].value_counts()

# Print the counts
print(class_counts)

ClassifyAs
Class2    6562
Class1    1779
Name: count, dtype: int64


In [786]:
#===========================================================================================================
#-------------------------Normal / Class Weight-------------------------------------------------------------
#===========================================================================================================

# # Split the data into features (X) and target (y)
# X = df.drop(columns=['ClassifyAs'])  # Features
# y = df['ClassifyAs']  # Target

# # Split the data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # # Define custom class weights
# # Total number of samples
# # total_samples = 3710 + 2962  #>90%

# # # Calculate class weights
# # weight_class_1 = total_samples / 3710
# # weight_class_2 = total_samples / 2962
# # weight_class_3 = total_samples / 751
# # weight_class_4 = total_samples / 3749

# # Total number of samples
# total_samples = 5235 + 1437    #tr89

# # Calculate class weights
# weight_class_1 = total_samples / 1437
# weight_class_2 = total_samples / 5235
# # weight_class_3 = total_samples / 751
# # weight_class_4 = total_samples / 3749

# # for classifier use Logloss, CrossEntropy, MultiClass, MultiClassOneVsAll
# # Create CatBoost classifier
# catboost_clf = CatBoostClassifier(iterations=1000,
#                                    depth=4, 
#                                    learning_rate=0.2,
# #                                    l2_leaf_reg=1,
#                                    loss_function='MultiClass',
# #                                    loss_function='Logloss',
#                                    eval_metric='Accuracy',
#                                    cat_features=['MaMH', 'Major', 'GVLT', 'GVTH'],  # Specify high cardinality categorical features
# #                                    auto_class_weights='Balanced',  # Automatically balance class weights
# #                                    class_weights=[weight_class_1, weight_class_2, weight_class_3, weight_class_4],  # Manually adjust class weights
#                                    class_weights=[weight_class_1, weight_class_2],  # Manually adjust class weights
#                                    random_seed=42)

# # Fit the model
# catboost_clf.fit(X_train, y_train)

# # Predict on the test set
# y_pred = catboost_clf.predict(X_test)

# accuracy = catboost_clf.score(X_test, y_test)
# print("Model Accuracy:", accuracy)

# # # Perform cross-validation
# cv_scores = cross_val_score(catboost_clf, X_train, y_train, cv=5, scoring='accuracy')  # You can adjust cv (number of folds) as needed

# # Print cross-validation scores
# print("Cross-Validation Scores:", cv_scores)
# print("Mean Accuracy:", np.mean(cv_scores))

#-------------Cross Validate-------------------------------------------------------------

# # Create CatBoost classifier
# catboost_clf = CatBoostClassifier(iterations=1000,
#                                    depth=4, 
#                                    learning_rate=0.2,
#                                    loss_function='MultiClass', 
#                                    eval_metric='Accuracy',
#                                    cat_features=['MaMH', 'Major', 'GVLT', 'GVTH'],  
#                                    random_seed=42)

# # Perform cross-validation
# cv_scores = cross_val_score(catboost_clf, X_train, y_train, cv=5, scoring='accuracy')  # You can adjust cv (number of folds) as needed

# # Print cross-validation scores
# print("Cross-Validation Scores:", cv_scores)
# print("Mean Accuracy:", np.mean(cv_scores))

#===========================================================================================================
#------------------SMOTE-N----------------------------------------------------------------------------------
#===========================================================================================================

# Step 1: Split the dataset into features (X) and target variable (y)
X = df.drop(columns=['ClassifyAs'])  # Features
y = df['ClassifyAs']  # Target

# Step 2: Split the features and target into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Apply SMOTE-N to balance the training data
categorical_indices = [index for index, column in enumerate(X_train.columns) if column in ['MaMH', 'Major', 'GVLT', 'GVTH']]
smote_nc = SMOTENC(categorical_features=categorical_indices, random_state=42)
X_resampled, y_resampled = smote_nc.fit_resample(X_train, y_train)

# Step 4: Train the model on the balanced training data
catboost_clf = CatBoostClassifier(iterations=1000,
                                   depth=4, 
                                   learning_rate=0.2,
                                   loss_function='MultiClass',
#                                    loss_function='Logloss',
#                                    grow_policy='Lossguide',
                                   eval_metric='Accuracy',
                                   cat_features=['MaMH', 'Major', 'GVLT', 'GVTH'],  
                                   random_seed=42)
catboost_clf.fit(X_resampled, y_resampled)

# Step 5: Predict on the testing data
y_pred = catboost_clf.predict(X_test)

# Step 6: Evaluate the model
accuracy = catboost_clf.score(X_test, y_test)
print("Model Accuracy:", accuracy)

# Perform cross-validation
cv_scores = cross_val_score(catboost_clf, X_train_resampled, y_train_resampled, cv=5, scoring='accuracy')  # You can adjust cv (number of folds) as needed

# Print cross-validation scores
print("Cross-Validation Scores:", cv_scores)
print("Mean Accuracy:", np.mean(cv_scores))

#===========================================================================================================
#------------------Oversampling Minor Class (2 Classes)-----------------------------------------------------
#===========================================================================================================

# # Split the data into features (X) and target (y)
# X = df.drop(columns=['ClassifyAs'])  # Features
# y = df['ClassifyAs']  # Target

# # Split the data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Combine the features and target into a single DataFrame for the training data
# train_data = pd.concat([X_train, y_train], axis=1)

# # Separate majority and minority classes
# majority_class = train_data[train_data['ClassifyAs'] == 'Class2'] 
# minority_class = train_data[train_data['ClassifyAs'] == 'Class1']

# # Upsample minority class
# minority_upsampled = resample(minority_class,
#                               replace=True,     # Sample with replacement
#                               n_samples=len(majority_class),    # Match number of samples in majority class
#                               random_state=42)  # Set random state for reproducibility

# # Combine majority class and upsampled minority class
# train_upsampled = pd.concat([majority_class, minority_upsampled])

# # Separate features and target after oversampling
# X_train_resampled = train_upsampled.drop(columns=['ClassifyAs'])
# y_train_resampled = train_upsampled['ClassifyAs']

# # Create CatBoost classifier with balanced class weights
# # catboost_clf = CatBoostClassifier(iterations=5000,
# #                                    depth=6, 
# #                                    learning_rate=0.1,
# #                                    loss_function='Logloss',
# #                                    eval_metric='Accuracy',
# #                                    cat_features=['MaMH', 'Major', 'GVLT', 'GVTH'],  # Specify high cardinality categorical features
# #                                    auto_class_weights='Balanced',  # Automatically balance class weights
# #                                    random_seed=42)

# catboost_clf = CatBoostClassifier(iterations=1000,
#                                    depth=4, 
#                                    learning_rate=0.2,
#                                    loss_function='MultiClass',
#                                    eval_metric='Accuracy',
#                                    cat_features=['MaMH', 'Major', 'GVLT', 'GVTH'],  # Specify high cardinality categorical features
#                                    auto_class_weights='Balanced',  # Automatically balance class weights
#                                    random_seed=42)

# # Fit the model on the resampled data
# catboost_clf.fit(X_train_resampled, y_train_resampled)

# # Predict on the test set
# y_pred = catboost_clf.predict(X_test)

# # Evaluate model accuracy
# accuracy = catboost_clf.score(X_test, y_test)
# print("Model Accuracy:", accuracy)

# # # Perform cross-validation
# cv_scores = cross_val_score(catboost_clf, X_train_resampled, y_train_resampled, cv=5, scoring='accuracy')  # You can adjust cv (number of folds) as needed

# # Print cross-validation scores
# print("Cross-Validation Scores:", cv_scores)
# print("Mean Accuracy:", np.mean(cv_scores))

#===========================================================================================================
#------------------Oversampling Minor Class (4 Classes)-----------------------------------------------------
#===========================================================================================================

# # Split the data into features (X) and target (y)
# X = df.drop(columns=['ClassifyAs'])  # Features
# y = df['ClassifyAs']  # Target

# # Split the data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=29)

# # Combine the features and target into a single DataFrame for the training data
# train_data = pd.concat([X_train, y_train], axis=1)

# # Separate majority and minority classes
# majority_class = train_data[train_data['ClassifyAs'] == 'Class4']  # Assuming 'Class4' is the majority class
# minority_classes = [train_data[train_data['ClassifyAs'] == cls] for cls in ['Class1', 'Class2', 'Class3']]

# # Upsample each minority class individually
# minority_upsampled = pd.concat([resample(minority_class, replace=True, n_samples=len(majority_class), random_state=42)
#                                 for minority_class in minority_classes])

# # Combine majority class and upsampled minority classes
# train_upsampled = pd.concat([majority_class] + [minority_upsampled])

# # Separate features and target after oversampling
# X_train_resampled = train_upsampled.drop(columns=['ClassifyAs'])
# y_train_resampled = train_upsampled['ClassifyAs']

# # Create CatBoost classifier with balanced class weights
# catboost_clf = CatBoostClassifier(iterations=5000,
#                                   depth=6,
#                                   learning_rate=0.1,
#                                   loss_function='MultiClass',
#                                   eval_metric='Accuracy',
#                                   cat_features=['MaMH', 'Major', 'GVLT', 'GVTH'],  # Specify high cardinality categorical features
#                                   auto_class_weights='Balanced',  # Automatically balance class weights
#                                   random_seed=42)

# # Fit the model on the resampled data
# catboost_clf.fit(X_train_resampled, y_train_resampled)

# # Predict on the test set
# y_pred = catboost_clf.predict(X_test)

# # Evaluate model accuracy
# accuracy = catboost_clf.score(X_test, y_test)
# print("Model Accuracy:", accuracy)

0:	learn: 0.6811843	total: 24.5ms	remaining: 24.5s
1:	learn: 0.7412607	total: 52.8ms	remaining: 26.4s
2:	learn: 0.7469914	total: 68.1ms	remaining: 22.6s
3:	learn: 0.7506208	total: 84.2ms	remaining: 21s
4:	learn: 0.7614136	total: 99.5ms	remaining: 19.8s
5:	learn: 0.7723973	total: 115ms	remaining: 19s
6:	learn: 0.7768863	total: 129ms	remaining: 18.2s
7:	learn: 0.7791786	total: 142ms	remaining: 17.7s
8:	learn: 0.7811843	total: 156ms	remaining: 17.2s
9:	learn: 0.7827125	total: 170ms	remaining: 16.9s
10:	learn: 0.7850048	total: 184ms	remaining: 16.5s
11:	learn: 0.7859599	total: 200ms	remaining: 16.4s
12:	learn: 0.7888252	total: 213ms	remaining: 16.2s
13:	learn: 0.7915950	total: 227ms	remaining: 16s
14:	learn: 0.7924546	total: 244ms	remaining: 16s
15:	learn: 0.7930277	total: 263ms	remaining: 16.1s
16:	learn: 0.7943649	total: 280ms	remaining: 16.2s
17:	learn: 0.7948424	total: 297ms	remaining: 16.2s
18:	learn: 0.7973257	total: 312ms	remaining: 16.1s
19:	learn: 0.7971347	total: 328ms	remaining:

173:	learn: 0.8387775	total: 2.32s	remaining: 11s
174:	learn: 0.8395415	total: 2.34s	remaining: 11s
175:	learn: 0.8395415	total: 2.35s	remaining: 11s
176:	learn: 0.8395415	total: 2.36s	remaining: 11s
177:	learn: 0.8395415	total: 2.38s	remaining: 11s
178:	learn: 0.8400191	total: 2.39s	remaining: 11s
179:	learn: 0.8404967	total: 2.4s	remaining: 10.9s
180:	learn: 0.8407832	total: 2.41s	remaining: 10.9s
181:	learn: 0.8410697	total: 2.42s	remaining: 10.9s
182:	learn: 0.8407832	total: 2.44s	remaining: 10.9s
183:	learn: 0.8411652	total: 2.45s	remaining: 10.9s
184:	learn: 0.8410697	total: 2.46s	remaining: 10.8s
185:	learn: 0.8407832	total: 2.47s	remaining: 10.8s
186:	learn: 0.8405922	total: 2.49s	remaining: 10.8s
187:	learn: 0.8409742	total: 2.5s	remaining: 10.8s
188:	learn: 0.8417383	total: 2.51s	remaining: 10.8s
189:	learn: 0.8424069	total: 2.52s	remaining: 10.8s
190:	learn: 0.8425979	total: 2.54s	remaining: 10.7s
191:	learn: 0.8426934	total: 2.55s	remaining: 10.7s
192:	learn: 0.8424069	tota

338:	learn: 0.8604585	total: 4.67s	remaining: 9.11s
339:	learn: 0.8600764	total: 4.69s	remaining: 9.1s
340:	learn: 0.8598854	total: 4.7s	remaining: 9.09s
341:	learn: 0.8597899	total: 4.72s	remaining: 9.08s
342:	learn: 0.8601719	total: 4.74s	remaining: 9.07s
343:	learn: 0.8602674	total: 4.75s	remaining: 9.06s
344:	learn: 0.8608405	total: 4.76s	remaining: 9.05s
345:	learn: 0.8607450	total: 4.78s	remaining: 9.04s
346:	learn: 0.8607450	total: 4.79s	remaining: 9.02s
347:	learn: 0.8606495	total: 4.81s	remaining: 9.02s
348:	learn: 0.8605540	total: 4.83s	remaining: 9s
349:	learn: 0.8602674	total: 4.84s	remaining: 9s
350:	learn: 0.8610315	total: 4.86s	remaining: 8.98s
351:	learn: 0.8611270	total: 4.87s	remaining: 8.97s
352:	learn: 0.8611270	total: 4.89s	remaining: 8.96s
353:	learn: 0.8612225	total: 4.9s	remaining: 8.95s
354:	learn: 0.8609360	total: 4.92s	remaining: 8.94s
355:	learn: 0.8607450	total: 4.93s	remaining: 8.93s
356:	learn: 0.8610315	total: 4.95s	remaining: 8.92s
357:	learn: 0.8609360

499:	learn: 0.8710602	total: 6.98s	remaining: 6.98s
500:	learn: 0.8709647	total: 6.99s	remaining: 6.97s
501:	learn: 0.8708691	total: 7.01s	remaining: 6.95s
502:	learn: 0.8706781	total: 7.02s	remaining: 6.94s
503:	learn: 0.8704871	total: 7.03s	remaining: 6.92s
504:	learn: 0.8707736	total: 7.04s	remaining: 6.91s
505:	learn: 0.8709647	total: 7.06s	remaining: 6.89s
506:	learn: 0.8711557	total: 7.07s	remaining: 6.88s
507:	learn: 0.8710602	total: 7.08s	remaining: 6.86s
508:	learn: 0.8714422	total: 7.1s	remaining: 6.85s
509:	learn: 0.8709647	total: 7.11s	remaining: 6.83s
510:	learn: 0.8708691	total: 7.12s	remaining: 6.82s
511:	learn: 0.8706781	total: 7.14s	remaining: 6.8s
512:	learn: 0.8708691	total: 7.15s	remaining: 6.79s
513:	learn: 0.8707736	total: 7.17s	remaining: 6.78s
514:	learn: 0.8706781	total: 7.18s	remaining: 6.76s
515:	learn: 0.8708691	total: 7.19s	remaining: 6.75s
516:	learn: 0.8708691	total: 7.21s	remaining: 6.73s
517:	learn: 0.8707736	total: 7.22s	remaining: 6.72s
518:	learn: 0.

661:	learn: 0.8789876	total: 9.11s	remaining: 4.65s
662:	learn: 0.8789876	total: 9.12s	remaining: 4.64s
663:	learn: 0.8789876	total: 9.14s	remaining: 4.62s
664:	learn: 0.8790831	total: 9.15s	remaining: 4.61s
665:	learn: 0.8790831	total: 9.16s	remaining: 4.59s
666:	learn: 0.8794651	total: 9.18s	remaining: 4.58s
667:	learn: 0.8795606	total: 9.19s	remaining: 4.57s
668:	learn: 0.8794651	total: 9.2s	remaining: 4.55s
669:	learn: 0.8799427	total: 9.22s	remaining: 4.54s
670:	learn: 0.8799427	total: 9.23s	remaining: 4.53s
671:	learn: 0.8802292	total: 9.24s	remaining: 4.51s
672:	learn: 0.8806113	total: 9.26s	remaining: 4.5s
673:	learn: 0.8806113	total: 9.27s	remaining: 4.48s
674:	learn: 0.8807068	total: 9.28s	remaining: 4.47s
675:	learn: 0.8809933	total: 9.29s	remaining: 4.45s
676:	learn: 0.8811843	total: 9.31s	remaining: 4.44s
677:	learn: 0.8810888	total: 9.32s	remaining: 4.43s
678:	learn: 0.8807068	total: 9.34s	remaining: 4.41s
679:	learn: 0.8808978	total: 9.35s	remaining: 4.4s
680:	learn: 0.8

...

303:	learn: 0.8324976	total: 4.19s	remaining: 9.59s
304:	learn: 0.8330946	total: 4.2s	remaining: 9.58s
305:	learn: 0.8326170	total: 4.22s	remaining: 9.57s
306:	learn: 0.8327364	total: 4.23s	remaining: 9.55s
307:	learn: 0.8329752	total: 4.24s	remaining: 9.53s
308:	learn: 0.8333333	total: 4.25s	remaining: 9.52s
309:	learn: 0.8336915	total: 4.27s	remaining: 9.5s
310:	learn: 0.8336915	total: 4.28s	remaining: 9.49s
311:	learn: 0.8341691	total: 4.29s	remaining: 9.47s
312:	learn: 0.8338109	total: 4.31s	remaining: 9.45s
313:	learn: 0.8341691	total: 4.32s	remaining: 9.44s
314:	learn: 0.8346466	total: 4.33s	remaining: 9.42s
315:	learn: 0.8348854	total: 4.34s	remaining: 9.4s
316:	learn: 0.8347660	total: 4.36s	remaining: 9.39s
317:	learn: 0.8352436	total: 4.37s	remaining: 9.37s
318:	learn: 0.8357211	total: 4.38s	remaining: 9.36s
319:	learn: 0.8356017	total: 4.4s	remaining: 9.35s
320:	learn: 0.8356017	total: 4.41s	remaining: 9.33s
321:	learn: 0.8353629	total: 4.42s	remaining: 9.32s
322:	learn: 0.83

470:	learn: 0.8505253	total: 6.36s	remaining: 7.14s
471:	learn: 0.8511223	total: 6.37s	remaining: 7.13s
472:	learn: 0.8511223	total: 6.39s	remaining: 7.12s
473:	learn: 0.8511223	total: 6.4s	remaining: 7.1s
474:	learn: 0.8512416	total: 6.41s	remaining: 7.09s
475:	learn: 0.8506447	total: 6.43s	remaining: 7.07s
476:	learn: 0.8506447	total: 6.44s	remaining: 7.06s
477:	learn: 0.8506447	total: 6.45s	remaining: 7.04s
478:	learn: 0.8510029	total: 6.46s	remaining: 7.03s
479:	learn: 0.8508835	total: 6.47s	remaining: 7.01s
480:	learn: 0.8508835	total: 6.49s	remaining: 7s
481:	learn: 0.8508835	total: 6.5s	remaining: 6.99s
482:	learn: 0.8505253	total: 6.51s	remaining: 6.97s
483:	learn: 0.8506447	total: 6.53s	remaining: 6.96s
484:	learn: 0.8506447	total: 6.54s	remaining: 6.94s
485:	learn: 0.8508835	total: 6.55s	remaining: 6.93s
486:	learn: 0.8508835	total: 6.56s	remaining: 6.91s
487:	learn: 0.8508835	total: 6.58s	remaining: 6.9s
488:	learn: 0.8505253	total: 6.59s	remaining: 6.89s
489:	learn: 0.85064

630:	learn: 0.8634193	total: 8.46s	remaining: 4.95s
631:	learn: 0.8635387	total: 8.47s	remaining: 4.93s
632:	learn: 0.8637775	total: 8.48s	remaining: 4.92s
633:	learn: 0.8641356	total: 8.5s	remaining: 4.9s
634:	learn: 0.8640162	total: 8.51s	remaining: 4.89s
635:	learn: 0.8642550	total: 8.52s	remaining: 4.88s
636:	learn: 0.8641356	total: 8.53s	remaining: 4.86s
637:	learn: 0.8644938	total: 8.54s	remaining: 4.85s
638:	learn: 0.8642550	total: 8.56s	remaining: 4.83s
639:	learn: 0.8643744	total: 8.57s	remaining: 4.82s
640:	learn: 0.8641356	total: 8.58s	remaining: 4.81s
641:	learn: 0.8636581	total: 8.6s	remaining: 4.79s
642:	learn: 0.8638968	total: 8.61s	remaining: 4.78s
643:	learn: 0.8641356	total: 8.62s	remaining: 4.76s
644:	learn: 0.8638968	total: 8.64s	remaining: 4.75s
645:	learn: 0.8638968	total: 8.65s	remaining: 4.74s
646:	learn: 0.8638968	total: 8.66s	remaining: 4.72s
647:	learn: 0.8631805	total: 8.67s	remaining: 4.71s
648:	learn: 0.8636581	total: 8.69s	remaining: 4.7s
649:	learn: 0.86

791:	learn: 0.8715377	total: 10.6s	remaining: 2.77s
792:	learn: 0.8714183	total: 10.6s	remaining: 2.76s
793:	learn: 0.8709408	total: 10.6s	remaining: 2.75s
794:	learn: 0.8709408	total: 10.6s	remaining: 2.73s
795:	learn: 0.8710602	total: 10.6s	remaining: 2.72s
796:	learn: 0.8709408	total: 10.6s	remaining: 2.71s
797:	learn: 0.8712989	total: 10.6s	remaining: 2.69s
798:	learn: 0.8715377	total: 10.7s	remaining: 2.68s
799:	learn: 0.8714183	total: 10.7s	remaining: 2.67s
800:	learn: 0.8715377	total: 10.7s	remaining: 2.65s
801:	learn: 0.8716571	total: 10.7s	remaining: 2.64s
802:	learn: 0.8717765	total: 10.7s	remaining: 2.63s
803:	learn: 0.8718959	total: 10.7s	remaining: 2.61s
804:	learn: 0.8717765	total: 10.7s	remaining: 2.6s
805:	learn: 0.8715377	total: 10.7s	remaining: 2.59s
806:	learn: 0.8717765	total: 10.8s	remaining: 2.57s
807:	learn: 0.8717765	total: 10.8s	remaining: 2.56s
808:	learn: 0.8720153	total: 10.8s	remaining: 2.55s
809:	learn: 0.8724928	total: 10.8s	remaining: 2.53s
810:	learn: 0

960:	learn: 0.8815664	total: 12.9s	remaining: 523ms
961:	learn: 0.8816858	total: 12.9s	remaining: 510ms
962:	learn: 0.8818052	total: 12.9s	remaining: 496ms
963:	learn: 0.8816858	total: 12.9s	remaining: 483ms
964:	learn: 0.8814470	total: 12.9s	remaining: 469ms
965:	learn: 0.8821633	total: 12.9s	remaining: 456ms
966:	learn: 0.8818052	total: 13s	remaining: 442ms
967:	learn: 0.8818052	total: 13s	remaining: 429ms
968:	learn: 0.8825215	total: 13s	remaining: 416ms
969:	learn: 0.8822827	total: 13s	remaining: 402ms
970:	learn: 0.8825215	total: 13s	remaining: 389ms
971:	learn: 0.8828797	total: 13s	remaining: 375ms
972:	learn: 0.8827603	total: 13s	remaining: 362ms
973:	learn: 0.8831184	total: 13.1s	remaining: 349ms
974:	learn: 0.8829990	total: 13.1s	remaining: 335ms
975:	learn: 0.8828797	total: 13.1s	remaining: 322ms
976:	learn: 0.8832378	total: 13.1s	remaining: 308ms
977:	learn: 0.8837154	total: 13.1s	remaining: 295ms
978:	learn: 0.8839542	total: 13.1s	remaining: 281ms
979:	learn: 0.8840735	tota

In [787]:
X_test.head(5)

,MaMH,Major,SoTinChi,SiSoTKB,TKB_LT2,TKB_LT3,TKB_LT4,TKB_LT5,TKB_LT6,TKB_LT7,TKB_TH2,TKB_TH3,TKB_TH4,TKB_TH5,TKB_TH6,TKB_TH7,HK,GVLT,GVTH
7023,PE017IU,IU PE,2,90.0,0,0,0,0,0,3,0,0,0,0,0,0,1,Pham Dinh Huan,unassigned
5201,MAFE208IU,MA MA,4,50.0,7,0,0,0,0,0,0,0,0,0,0,0,2,Mai Duc Thanh,unassigned
4280,IS056IU,IEMIEM,1,25.0,0,0,0,0,0,0,0,0,0,0,7,0,2,unassigned,Nguyen Hoang An
940,CH012IU,BT BT,1,20.0,7,0,0,0,0,0,0,0,0,0,0,0,3,Le Nguyen Thien Phuc,unassigned
5364,BA130IU,BA BA,3,60.0,0,0,0,0,0,1,0,0,0,0,0,0,1,Nguyen Tran Nguyen Khai,unassigned


In [788]:
X_test.dtypes

MaMH         object
Major        object
SoTinChi      int64
SiSoTKB     float64
TKB_LT2       int64
TKB_LT3       int64
TKB_LT4       int64
TKB_LT5       int64
TKB_LT6       int64
TKB_LT7       int64
TKB_TH2       int64
TKB_TH3       int64
TKB_TH4       int64
TKB_TH5       int64
TKB_TH6       int64
TKB_TH7       int64
HK            int64
GVLT         object
GVTH         object
dtype: object

In [789]:
X_test.shape

(1669, 19)

In [790]:
# # Define the parameter grid
# param_grid = {
#     'iterations': [500, 1000],  # Number of boosting iterations
#     'depth': [4, 6, 8],          # Maximum depth of each tree
#     'learning_rate': [0.05, 0.1],# Learning rate
#     # Add more parameters to tune if needed
# }

# # Create CatBoost classifier
# catboost_clf = CatBoostClassifier(loss_function='Logloss',
#                                   eval_metric='Accuracy',
#                                   cat_features=['MaMH', 'Major', 'GVLT', 'GVTH'],
#                                   auto_class_weights='Balanced',
#                                   random_seed=42)

# # Perform grid search
# grid_search = GridSearchCV(estimator=catboost_clf, param_grid=param_grid, cv=5, scoring='accuracy')
# grid_search.fit(X_train, y_train)

# # Best parameters found
# print("Best parameters:", grid_search.best_params_)

# # Best score
# print("Best accuracy:", grid_search.best_score_)

# # Predict on the test set using the best model
# best_clf = grid_search.best_estimator_
# y_pred = best_clf.predict(X_test)

# # Calculate accuracy
# accuracy = best_clf.score(X_test, y_test)
# print("Model Accuracy:", accuracy)

In [791]:
# # Convert y_resampled to a pandas Series
y_train_series = pd.Series(y_train)

# Count the occurrences of each class
class_counts2 = y_train_series.value_counts()

# Print the class counts
print("Class Counts in y_train_series:")
print(class_counts2)

Class Counts in y_train_series:
ClassifyAs
Class2    5235
Class1    1437
Name: count, dtype: int64


In [792]:
# # Convert y_resampled to a pandas Series
y_test_series = pd.Series(y_test)

# Count the occurrences of each class
class_counts3 = y_test_series.value_counts()

# Print the class counts
print("Class Counts in y_test_series:")
print(class_counts3)

Class Counts in y_test_series:
ClassifyAs
Class2    1327
Class1     342
Name: count, dtype: int64


In [793]:
# # Convert y_resampled to a pandas Series
# # y_resampled_series = pd.Series(y_resampled)
# y_resampled_series = pd.Series(y_train_resampled)

# # Count the occurrences of each class
# class_counts = y_resampled_series.value_counts()

# # Print the class counts
# print("Class Counts in y_resampled:")
# print(class_counts)

Class Counts in y_resampled:
ClassifyAs
Class2    5235
Class1    5235
Name: count, dtype: int64


In [794]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Create a DataFrame for actual vs. predicted values
results_df = pd.DataFrame({'Actual': y_test.values.flatten(), 'Predicted': y_pred.flatten()})
results_df

Accuracy: 0.7711204313960456


,Actual,Predicted
0,Class2,Class2
1,Class2,Class2
2,Class2,Class2
3,Class2,Class2
4,Class2,Class2
5,Class2,Class2
6,Class2,Class2
7,Class2,Class2
8,Class2,Class2
9,Class1,Class2


In [795]:
# Create a DataFrame to count total correct and incorrect predictions for each class
class_counts = {}

# Update counts based on actual vs. predicted values
for actual, predicted in zip(y_test, y_pred):
    if actual not in class_counts:
        class_counts[actual] = {'Correct': 0, 'Incorrect': 0}
    if actual == predicted:
        class_counts[actual]['Correct'] += 1
    else:
        class_counts[actual]['Incorrect'] += 1

# Convert dictionary to DataFrame
class_count_df = pd.DataFrame(class_counts).transpose().reset_index()
class_count_df.rename(columns={'index': 'Class'}, inplace=True)

# Calculate total correct and incorrect predictions across all classes
total_correct_all_classes = class_count_df['Correct'].sum()
total_incorrect_all_classes = class_count_df['Incorrect'].sum()

# Add a row for the total counts across all classes
total_row = pd.DataFrame({'Class': 'Total', 'Correct': total_correct_all_classes, 'Incorrect': total_incorrect_all_classes}, index=[0])
class_count_df = pd.concat([class_count_df, total_row], ignore_index=True)

# Display the DataFrame
print("\nClass-wise Correct and Incorrect Predictions:")
print(class_count_df)


Class-wise Correct and Incorrect Predictions:
    Class  Correct  Incorrect
0  Class2     1132        195
1  Class1      155        187
2   Total     1287        382


In [796]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

# # Classification Report
# print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Confusion Matrix:
[[ 155  187]
 [ 195 1132]]
              precision    recall  f1-score   support

      Class1       0.44      0.45      0.45       342
      Class2       0.86      0.85      0.86      1327

    accuracy                           0.77      1669
   macro avg       0.65      0.65      0.65      1669
weighted avg       0.77      0.77      0.77      1669



In [797]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.7711204313960456
Precision: 0.7731114141923391
Recall: 0.7711204313960456
F1-score: 0.7720974338232061


In [798]:
# # Define the filename to save your model
# model_filename = 'dhqt_90slots_classifier_model.cbm'
# # model_filename = 'dhqt_tr89_classifier_model.cbm'

# # Save the model
# catboost_clf.save_model(model_filename)

# print("Model saved successfully as", model_filename)